In [1]:
# Install decord and other video processing libraries
!pip install decord
!pip install av
!pip install pyav
!pip install opencv-python
!pip install moviepy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
ERROR: Could not find a version that satisfies the requirement pyav (from versions: none)
ERROR: No matching distribution found for pyav
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import numpy as np
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
from huggingface_hub import snapshot_download, hf_hub_download
import bisect
import shutil
import numpy as np
from PIL import Image
import requests
import copy
import warnings
from decord import VideoReader, cpu
from datasets import load_dataset
import re

In [3]:
data_list = {
    "Action Sequence": ("action_sequence.json", "star/Charades_v1_480/", "video", True), # has start & end
    "Action Prediction": ("action_prediction.json", "star/Charades_v1_480/", "video", True), # has start & end
    "Action Antonym": ("action_antonym.json", "ssv2_video/", "video", False),
    "Fine-grained Action": ("fine_grained_action.json", "Moments_in_Time_Raw/videos/", "video", False),
    "Unexpected Action": ("unexpected_action.json", "FunQA_test/test/", "video", False),
    "Object Existence": ("object_existence.json", "clevrer/video_validation/", "video", False),
    "Object Interaction": ("object_interaction.json", "star/Charades_v1_480/", "video", True), # has start & end
    "Object Shuffle": ("object_shuffle.json", "perception/videos/", "video", False),
    "Moving Direction": ("moving_direction.json", "clevrer/video_validation/", "video", False),
    "Action Localization": ("action_localization.json", "sta/sta_video/", "video", True),  # has start & end
    "Scene Transition": ("scene_transition.json", "scene_qa/video/", "video", False),
    "Action Count": ("action_count.json", "perception/videos/", "video", False),
    "Moving Count": ("moving_count.json", "clevrer/video_validation/", "video", False),
    "Moving Attribute": ("moving_attribute.json", "clevrer/video_validation/", "video", False),
    "State Change": ("state_change.json", "perception/videos/", "video", False),
    "Fine-grained Pose": ("fine_grained_pose.json", "nturgbd/", "video", False),
    "Character Order": ("character_order.json", "perception/videos/", "video", False),
    "Egocentric Navigation": ("egocentric_navigation.json", "vlnqa/", "video", False),
    "Episodic Reasoning": ("episodic_reasoning.json", "tvqa/frames_fps3_hq/", "frame", True),  # has start & end, read frame
    "Counterfactual Inference": ("counterfactual_inference.json", "clevrer/video_validation/", "video", False),
}

data_dir = "dataset"
if not os.path.exists(data_dir):
    os.mkdir("dataset")

def read_video_pyav(video_path, start, end, n_frames=8):
    """
    Reads a video for given start-end timestamps interval
    and uniformly samples 8 frames of it
    """
    container = av.open(video_path)
    video = container.streams.get(0)[0]

    av_timestamps = [
        int(packet.pts * video.time_base) for packet in container.demux(video) if packet.pts is not None
    ]

    av_timestamps.sort()
    start_id = bisect.bisect_left(av_timestamps, start)
    end_id = bisect.bisect_left(av_timestamps, end)

    # in case it is a very short video, lets take a longer duration and sample
    if end_id  - start_id < 10:
        end_id += 10
        start_id -= 10

    end_id = min(len(av_timestamps) - 1, end_id)
    start_id = max(1, start_id)

    # We sample n_frames frames for tuning following the original paper
    # But we can increase the number of frames for longer videos and check out if it helps performance
    # Change the below "n_frames" to any number of frames you want, and note that more frames -> more computational resources needed
    indices = np.linspace(start_id, end_id, n_frames).astype(int)

    frames = []
    container.seek(0)
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_id:
            break
        if i >= start_id and i in indices:
            frames.append(frame)
    assert len(frames) == n_frames, f"Got {len(frames)} frames but should be {n_frames}. Check the indices: {indices};, start_id: {start_id}, end_id: {end_id}. Len of video is {len(av_timestamps)} frames."
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

def collate_read_video(example, path):
    # Some datasets have a start-end interval, so we try to get it if exists.
    # Otherwise just set a very large end timestamp
    clip = read_video_pyav(f'{path}/{example["video"]}', example.get("start", 1), example.get("end", 1e+10))
    example["clip"] = clip
    return example

TASK_NAME = "Moving Count"
annotation_fn, video_dir, video_type, has_clip = data_list[TASK_NAME]

annotation_fn_local = hf_hub_download(
    repo_id="OpenGVLab/MVBench",
    filename='json/' + annotation_fn,
    repo_type="dataset",
    local_dir=data_dir
)

video_zip_name = video_dir.split("/")[0] + ".zip"
videos_zip = hf_hub_download(
    repo_id="OpenGVLab/MVBench",
    filename='video/' + video_zip_name,
    repo_type="dataset",
    local_dir=data_dir
)

for zip_file in os.listdir(f"{data_dir}/video"):
    if zip_file.endswith(".zip"):
        shutil.unpack_archive(
            f"{data_dir}/video/{zip_file}",
            f"{data_dir}/video/videos_unzipped/"
        )

ds = load_dataset("json", data_files=annotation_fn_local, split="train")
ds

has_missing = False
for sample in ds:
    if not os.path.exists(f"{data_dir}/video/videos_unzipped/{video_dir}/{sample['video']}"):
        print(f"Video `{sample['video']}` does not exists!")
        has_missing = True

print(f"Dataset length = {len(ds)}")
if has_missing:
    ds = ds.filter(lambda x: os.path.exists(f"{data_dir}/video/videos_unzipped/{video_dir}/{x['video']}"))

print(f"Dataset length = {len(ds)}")

moving_count.json: 0.00B [00:00, ?B/s]

video/clevrer.zip:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset length = 200
Dataset length = 200


In [4]:
ds

Dataset({
    features: ['video', 'question', 'candidates', 'answer'],
    num_rows: 200
})

In [5]:
from pathlib import Path

unzipped_dir = f"{data_dir}/video/videos_unzipped"
video_subdir = Path(video_dir).name
video_path = os.path.join(unzipped_dir, "star", video_subdir)

In [6]:
video_filename = ds[1]['video']
full_video_path = f"{unzipped_dir}/{video_dir}/{video_filename}"

In [7]:
prompt = ds[1]['question']

In [8]:
torch.cuda.empty_cache()

In [12]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.2 MB/s eta 0:00:00 0:00:01


In [18]:
import torch
from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration

model = LlavaOnevisionForConditionalGeneration.from_pretrained("llava-hf/llava-onevision-qwen2-7b-ov-hf", device_map="auto")
processor = AutoProcessor.from_pretrained("llava-hf/llava-onevision-qwen2-7b-ov-hf")

video_path = full_video_path
conversation = [
    {

        "role": "user",
        "content": [
            {"type": "video", "path": video_path},
            {"type": "text", "text": f"{prompt}\nAnswer with a number only"},
            ],
    },
]

inputs = processor.apply_chat_template(
    conversation,
    num_frames=8,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device, torch.float16)

out = model.generate(**inputs, max_new_tokens=60)
res = processor.batch_decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Unused or unrecognized kwargs: return_tensors.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


In [23]:
res[0]

'user \nHow many moving cylinders are there?\nAnswer with a number onlyassistant\n2'

In [25]:
match = re.search(r"assistant\s*(\d+)", res[0])
if match:
    number = int(match.group(1))
else:
    print("Number not found")

In [29]:
ds[1]['answer']

'0'

In [37]:
import time
from tqdm import tqdm

comp_time = 0
correct_guessed = 0

model.eval()
with torch.no_grad():

    for i in tqdm(range(len(ds)), desc="Processing samples"):
        sample = ds[i]
        video_filename = sample['video']
        full_video_path = f"{unzipped_dir}/{video_dir}/{video_filename}"
        prompt = sample['question']
        real_answer = sample['answer']
        question = f"{prompt}\nYou have the following candidates for the right answer: {sample['candidates'][0]}, {sample['candidates'][1]}, {sample['candidates'][2]}, {sample['candidates'][3]}.\Your correct answer must be one of the four presented candidates"
        video_path = full_video_path
        conversation = [
            {
        
                "role": "user",
                "content": [
                    {"type": "video", "path": video_path},
                    {"type": "text", "text": question},
                    ],
            },
        ]
        
        inputs = processor.apply_chat_template(
            conversation,
            num_frames=8,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(model.device, torch.float16)
        startt = time.time()
        
        out = model.generate(**inputs, max_new_tokens=60)
        res = processor.batch_decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        endt = time.time()
        elapsed_time = endt - startt
        comp_time += elapsed_time
        match = re.search(r"assistant\s*(\d+)", res[0])
        if match:
            number = int(match.group(1))
        else:
            print("Number not found")

        if int(number) == int(real_answer):
            correct_guessed +=1

accuracy = correct_guessed / len(ds)
print(f'Computational Time: {comp_time}\nAccuracy: {accuracy}')

Unused or unrecognized kwargs: return_tensors.:00<?, ?it/s]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:01<06:37,  2.00s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:03<06:31,  1.98s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:05<06:30,  1.98s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:07<06:26,  1.97s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:09<06:22,  1.96s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:11<06:21,  1.97s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognize

Computational Time: 354.62246561050415
Accuracy: 0.46


In [38]:
data_list = {
    "Action Sequence": ("action_sequence.json", "star/Charades_v1_480/", "video", True), # has start & end
    "Action Prediction": ("action_prediction.json", "star/Charades_v1_480/", "video", True), # has start & end
    "Action Antonym": ("action_antonym.json", "ssv2_video/", "video", False),
    "Fine-grained Action": ("fine_grained_action.json", "Moments_in_Time_Raw/videos/", "video", False),
    "Unexpected Action": ("unexpected_action.json", "FunQA_test/test/", "video", False),
    "Object Existence": ("object_existence.json", "clevrer/video_validation/", "video", False),
    "Object Interaction": ("object_interaction.json", "star/Charades_v1_480/", "video", True), # has start & end
    "Object Shuffle": ("object_shuffle.json", "perception/videos/", "video", False),
    "Moving Direction": ("moving_direction.json", "clevrer/video_validation/", "video", False),
    "Action Localization": ("action_localization.json", "sta/sta_video/", "video", True),  # has start & end
    "Scene Transition": ("scene_transition.json", "scene_qa/video/", "video", False),
    "Action Count": ("action_count.json", "perception/videos/", "video", False),
    "Moving Count": ("moving_count.json", "clevrer/video_validation/", "video", False),
    "Moving Attribute": ("moving_attribute.json", "clevrer/video_validation/", "video", False),
    "State Change": ("state_change.json", "perception/videos/", "video", False),
    "Fine-grained Pose": ("fine_grained_pose.json", "nturgbd/", "video", False),
    "Character Order": ("character_order.json", "perception/videos/", "video", False),
    "Egocentric Navigation": ("egocentric_navigation.json", "vlnqa/", "video", False),
    "Episodic Reasoning": ("episodic_reasoning.json", "tvqa/frames_fps3_hq/", "frame", True),  # has start & end, read frame
    "Counterfactual Inference": ("counterfactual_inference.json", "clevrer/video_validation/", "video", False),
}

data_dir = "dataset"
if not os.path.exists(data_dir):
    os.mkdir("dataset")

def read_video_pyav(video_path, start, end, n_frames=8):
    """
    Reads a video for given start-end timestamps interval
    and uniformly samples 8 frames of it
    """
    container = av.open(video_path)
    video = container.streams.get(0)[0]

    av_timestamps = [
        int(packet.pts * video.time_base) for packet in container.demux(video) if packet.pts is not None
    ]

    av_timestamps.sort()
    start_id = bisect.bisect_left(av_timestamps, start)
    end_id = bisect.bisect_left(av_timestamps, end)

    # in case it is a very short video, lets take a longer duration and sample
    if end_id  - start_id < 10:
        end_id += 10
        start_id -= 10

    end_id = min(len(av_timestamps) - 1, end_id)
    start_id = max(1, start_id)

    # We sample n_frames frames for tuning following the original paper
    # But we can increase the number of frames for longer videos and check out if it helps performance
    # Change the below "n_frames" to any number of frames you want, and note that more frames -> more computational resources needed
    indices = np.linspace(start_id, end_id, n_frames).astype(int)

    frames = []
    container.seek(0)
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_id:
            break
        if i >= start_id and i in indices:
            frames.append(frame)
    assert len(frames) == n_frames, f"Got {len(frames)} frames but should be {n_frames}. Check the indices: {indices};, start_id: {start_id}, end_id: {end_id}. Len of video is {len(av_timestamps)} frames."
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

def collate_read_video(example, path):
    # Some datasets have a start-end interval, so we try to get it if exists.
    # Otherwise just set a very large end timestamp
    clip = read_video_pyav(f'{path}/{example["video"]}', example.get("start", 1), example.get("end", 1e+10))
    example["clip"] = clip
    return example

TASK_NAME = "Action Sequence"
annotation_fn, video_dir, video_type, has_clip = data_list[TASK_NAME]

annotation_fn_local = hf_hub_download(
    repo_id="OpenGVLab/MVBench",
    filename='json/' + annotation_fn,
    repo_type="dataset",
    local_dir=data_dir
)

video_zip_name = video_dir.split("/")[0] + ".zip"
videos_zip = hf_hub_download(
    repo_id="OpenGVLab/MVBench",
    filename='video/' + video_zip_name,
    repo_type="dataset",
    local_dir=data_dir
)

for zip_file in os.listdir(f"{data_dir}/video"):
    if zip_file.endswith(".zip"):
        shutil.unpack_archive(
            f"{data_dir}/video/{zip_file}",
            f"{data_dir}/video/videos_unzipped/"
        )

ds = load_dataset("json", data_files=annotation_fn_local, split="train")
ds

has_missing = False
for sample in ds:
    if not os.path.exists(f"{data_dir}/video/videos_unzipped/{video_dir}/{sample['video']}"):
        print(f"Video `{sample['video']}` does not exists!")
        has_missing = True

print(f"Dataset length = {len(ds)}")
if has_missing:
    ds = ds.filter(lambda x: os.path.exists(f"{data_dir}/video/videos_unzipped/{video_dir}/{x['video']}"))

print(f"Dataset length = {len(ds)}")

Couldn't access the Hub to check for update but local file already exists. Defaulting to existing file. (error: (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 5124a9f3-49ee-49f7-b6d6-820e4c9717c1)'))


Video `EDXBD.mp4` does not exists!
Video `K47J5.mp4` does not exists!
Video `9MNZ5.mp4` does not exists!
Video `QXT9W.mp4` does not exists!
Video `ABHC6.mp4` does not exists!
Video `ALXUC.mp4` does not exists!
Video `BAUQE.mp4` does not exists!
Video `PHH6B.mp4` does not exists!
Video `MNC10.mp4` does not exists!
Video `W7CR5.mp4` does not exists!
Video `Q8UJ8.mp4` does not exists!
Video `X9WTR.mp4` does not exists!
Dataset length = 200
Dataset length = 188


In [39]:
from pathlib import Path

unzipped_dir = f"{data_dir}/video/videos_unzipped"
video_subdir = Path(video_dir).name
video_path = os.path.join(unzipped_dir, "star", video_subdir)

video_filename = ds[1]['video']
full_video_path = f"{unzipped_dir}/{video_dir}/{video_filename}"

prompt = ds[1]['question']

import gc

gc.collect()
torch.cuda.empty_cache()

In [40]:
video_path = full_video_path
question = f"{prompt}\nYou have the following candidates for the right answer:\nA: {ds[1]['candidates'][0]}, B: {ds[1]['candidates'][1]}, C: {ds[1]['candidates'][2]}, D: {ds[1]['candidates'][3]}\nChoose the letter corresponding to the correct answer"
conversation = [
    {

        "role": "user",
        "content": [
            {"type": "video", "path": video_path},
            {"type": "text", "text": question},
            ],
    },
]

inputs = processor.apply_chat_template(
    conversation,
    num_frames=8,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device, torch.float16)

out = model.generate(**inputs, max_new_tokens=60)
res = processor.batch_decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)

Unused or unrecognized kwargs: return_tensors.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


In [42]:
letters = ['A', 'B', 'C', 'D'] 

ans = ds[1]['answer']
index = ds[1]['candidates'].index(ans)
print(letters[index])

match = re.search(r"assistant\s*([A-Z])", res[0]) 

if match:
    letter = match.group(1)
    print("Extracted letter:", letter)
else:
    print("No uppercase letter found after 'Assistant:'")
print(str(letters[index]) == str(letter))
print(ds[1]['answer'], ds[1]['candidates'])

C
Extracted letter: C
True
Opened the closet/cabinet. ['Tidied up the table.', 'Took the phone/camera.', 'Opened the closet/cabinet.', 'Washed the table.']


In [43]:
import time
from tqdm import tqdm

comp_time = 0
correct_guessed = 0
letters = ['A', 'B', 'C', 'D']
model.eval()
with torch.no_grad():

    for i in tqdm(range(len(ds)), desc="Processing samples"):
        sample = ds[i]
        video_filename = sample['video']
        full_video_path = f"{unzipped_dir}/{video_dir}/{video_filename}"
        prompt = sample['question']
        real_answer = sample['answer']
        ind = sample['candidates'].index(real_answer)
        ans_let = letters[ind]
        question = f"{prompt}\nYou have the following candidates for the right answer:\nA: {ds[1]['candidates'][0]}, B: {ds[1]['candidates'][1]}, C: {ds[1]['candidates'][2]}, D: {ds[1]['candidates'][3]}\nChoose the letter corresponding to the correct answer"
        video_path = full_video_path
        conversation = [
            {
        
                "role": "user",
                "content": [
                    {"type": "video", "path": video_path},
                    {"type": "text", "text": question},
                    ],
            },
        ]
        
        inputs = processor.apply_chat_template(
            conversation,
            num_frames=8,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(model.device, torch.float16)
        
        startt = time.time()
        
        out = model.generate(**inputs, max_new_tokens=60)
        res = processor.batch_decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        endt = time.time()
        elapsed_time = endt - startt
        
        comp_time += elapsed_time

        match = re.search(r"assistant\s*([A-Z])", res[0]) 
        if match:
            letter = match.group(1)
        else:
            print("No uppercase letter found after 'Assistant:'")

        if str(ans_let) == str(letter):
            correct_guessed +=1

accuracy = correct_guessed / len(ds)
print(f'Computational Time: {comp_time}\nAccuracy: {accuracy}')

Unused or unrecognized kwargs: return_tensors.:00<?, ?it/s]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:02<06:49,  2.19s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:04<06:37,  2.14s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:06<06:42,  2.18s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:08<06:23,  2.08s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:10<06:43,  2.20s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognized kwargs: return_tensors.:13<06:54,  2.28s/it]
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Unused or unrecognize

Computational Time: 332.8227803707123
Accuracy: 0.2925531914893617
